In [ ]:
import numpy as np
import pickle
import os

seed = 2023

In [ ]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

In [ ]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [ ]:
%run ../load_anno_mi.py

In [ ]:
anno_mi.head()

In [ ]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

# Baseline: FFN baseline

Using the embeddings for the sentences directly in a FFN to predict the client talk type.

In [ ]:
num_epochs = 100
hidden_dim_sizes = [[64, 64], [128, 128], [256, 256], [512, 512]]
dropout_rates = [0.1]
learning_rates = [1e-3, 5e-4, 1e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [ ]:
kwargs = {
    "num_epochs": num_epochs,
    "x_data": sbert_embeddings[client_index],
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "hidden_dim_sizes": hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

In [ ]:
ffn_current_kfold, best_ffn_current_kfold, _, __ = ffn_hyperparameter_search(
    results_output=f"{output_dir}/ffn_current_focal_{gamma}_kfold.csv",
    **kwargs,
)

In [ ]:
ffn_current_kfold

In [ ]:
ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_ffn_current_kfold

In [ ]:
best_ffn_current_kfold["f1"].mean()

In [ ]:
best_ffn_current_kfold["precision"].mean()

In [ ]:
best_ffn_current_kfold["recall"].mean()

In [ ]:
np.stack(best_ffn_current_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_ffn_current_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_ffn_current_kfold["recall_scores"]).mean(axis=0)